[View in Colaboratory](https://colab.research.google.com/github/Kristytiki/Neural-Network/blob/master/231_tensorflow_basic.ipynb)

## 231 Project 0

Today
- (1) Assignment
- (2) TensorFlow
- (3) GPU
- (4) PyTorch vs. TensorFlow

**Static shape**

In [21]:
import tensorflow as tf
import numpy as np

x = tf.placeholder(dtype=tf.float32, shape=[64, 10])
print(x.shape)
x_val = np.zeros([64, 10])

with tf.Session() as sess:
  print(tf.shape(x).eval(feed_dict={x: x_val}))

(64, 10)
[64 10]


**Dynamic shape**

In [3]:
import tensorflow as tf

with tf.Session():
  x = tf.placeholder(dtype=tf.float32, shape=[4])
  print(x.get_shape())
  y, _ = tf.unique(x)
  print(y.get_shape())
  print(y.eval(feed_dict={x: [0,1,2,3]}).shape)
  print(y.eval(feed_dict={x: [0,0,0,0]}).shape)
  

(4,)
(?,)
(4,)
(1,)


In [23]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 10, 5))

print(x.shape)

x2 = tf.reshape(x, (tf.shape(x)[0], -1))

x2_shape = tf.shape(x2)
with tf.Session():
  print(x2_shape.eval(feed_dict={x: np.zeros((32, 10, 5))}))

(?, 10, 5)
[32 50]


**tf.Print()**

In [0]:
x = tf.placeholder(tf.int32, shape=[4])
y = tf.Print(x, [tf.shape(x)], message='x=', summarize=4)

with tf.Session():
  y.eval(feed_dict={x:[0,1,2,3]})

No output in Colab. Should work in PyCharm. Remedy below.

In [36]:
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/enijkamp/tensorflow-snippets/master/memory_util.py')
open('memory_util.py', 'wb').write(response.read())
import memory_util

x = tf.placeholder(tf.int32, shape=[4])
y = tf.Print(x, [tf.shape(x)], message='x=', summarize=4)

with memory_util.capture_stderr() as stderr:
  with tf.Session():
    y.eval(feed_dict={x:[0,1,2,3]})
                      
print(stderr.getvalue())

2018-10-10 03:40:46.816072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-10 03:40:46.816146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-10 03:40:46.816166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2018-10-10 03:40:46.816183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2018-10-10 03:40:46.816442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1103] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 9911 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
x=[4]



**Convolution**

In [29]:
import numpy as np

conv_w1 = tf.Variable(tf.zeros([5, 5, 3, 32]))
conv_b1 = tf.Variable(tf.zeros(32,))

# N H W C
x = tf.placeholder(dtype=tf.float32, shape=[64, 32, 32, 3])

# block 1
x1_1_pad = tf.pad(x, [[0, 0], [2, 2], [2, 2], [0, 0]], mode='CONSTANT', constant_values=0)
x1_2_conv = tf.nn.conv2d(x1_1_pad, conv_w1, [1, 1, 1, 1], padding='VALID') + conv_b1
x1_3_pad = tf.pad(x1_2_conv, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='CONSTANT', constant_values=0)
x1_4_pool = tf.nn.max_pool(x1_3_pad, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
x1_5_relu = tf.nn.relu(x1_4_pool)

x1_5_relu = tf.Print(x1_5_relu, [tf.shape(x1_5_relu)], summarize=4)

with memory_util.capture_stderr() as stderr:
  with tf.Session():
    tf.initializers.global_variables().run()
    x1_5_relu.eval(feed_dict={x: np.zeros([64, 32, 32, 3])})

print(stderr.getvalue()) 

2018-10-10 02:39:40.702528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-10 02:39:40.702593: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-10 02:39:40.702613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2018-10-10 02:39:40.702628: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2018-10-10 02:39:40.702927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1103] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 9911 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
[64 16 16 32]



**Convolution (2)**

input 7x7

filter 3x3

stride 1

padding 0

(W-F+2P)/S+1

In [30]:

W = 7
F = 3
P = 0
S = 1

x_in = tf.placeholder(dtype=tf.float32, shape=[1, W, W, 1])
x_out = tf.nn.conv2d(x_in, filter=tf.zeros([F, F, 1, 1]), strides=[1, S, S, 1], padding='VALID')
x_out_shape = tf.shape(x_out)

with tf.Session():
  print(x_out_shape.eval(feed_dict={x_in: np.zeros([1, W, W, 1])}))
  
  assert (x_out_shape.eval(feed_dict={x_in: np.zeros([1, W, W, 1])}) == [1, 5, 5, 1]).all()

  print((W-F+2*P)/S+1)

[1 5 5 1]
5.0


**Padding**

In [0]:
x = tf.constant([1., 2., 3., 4., 5., 6.])
x = tf.reshape(x, [1, 2, 3, 1])

valid_pad = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
same_pad = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

assert valid_pad.get_shape() == [1,1,1,1]
assert same_pad.get_shape() == [1,1,2,1]

In [0]:
x = tf.constant([1., 2., 3., 4., 5., 6.])
x = tf.reshape(x, [1, 2, 3, 1])

valid_pad = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='VALID')
same_pad = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

assert valid_pad.get_shape() == [1,1,2,1]
assert same_pad.get_shape() == [1,2,3,1]

**GPU**

For this collab notebook:

Set Runtime -> Change runtime type to "GPU"

In [33]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)
assert device_name == '/device:GPU:0'

/device:GPU:0


- Copy and paste your solution here
- set USE_GPU=True
- Run on GPU which is about 40x faster

In [34]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13557701016319533664
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10392928256
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12930440421430216492
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


**TF versus PyTorch**

TensorFlow:
- funding by Google
- static computational graph
- "Define and run"
- "eager" mode resembles PyTorch
- static graph might be advantageous for production enviroments
- TensorFlow 2.0 will resemble PyTorch
  
PyTorch:
- funding by Facebook AI
- dynamic computational graph
- "Define by run"
- debugging simplified
- Google supports PyTorch 1.0 on GCE

In [0]:
import tensorflow as tf

x = tf.Variable(tf.ones((1, 1)))
y = tf.pow(x, 2)
print(y)


# view in debugger -> symbolic
g1 = tf.gradients(y, x)[0]

with tf.Session():
    tf.initializers.global_variables().run()
    print(g1.eval())

In [0]:
import torch

x = torch.ones(1, 1, requires_grad=True)
y = x ** 2
print(y)
g1 = torch.autograd.grad(y, x)[0]
print(g1)

# view in debugger -> numpy
g1_np = g1.cpu().numpy()

x = torch.ones(1, 1, requires_grad=True)
y = x ** 2
y.backward()
g2 = x.grad
print(g2)

assert g1 == g2